In [ ]:
import time
import os

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

from tqdm.autonotebook import tqdm
from torch.utils.data import Dataset
from torchvision import transforms
from scipy.spatial.distance import cdist

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
img_size = 68

epochs = 2000
batch_size = 32
learning_rate = 0.258

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform = None):
        self.img = img_dir
        self.mask = mask_dir
        self.transform = transform
        self.imgpath = os.listdir(img_dir)
        self.mskpath = os.listdir(mask_dir)

    def __len__(self):
        return len(self.imgpath)
        
    
    def __getitem__(self, index):
        img_path = os.path.join(self.img, self.imgpath[index])
        mask_path = os.path.join(self.mask, self.mskpath[index])

        image = cv.resize(cv.imread(img_path,cv.IMREAD_GRAYSCALE),(img_size,img_size))
        mask = cv.resize(cv.imread(mask_path,cv.IMREAD_GRAYSCALE),(img_size,img_size)) 
        
        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return (image, mask)

In [ ]:
path_data_train = "C:/Users/sonbu/Desktop/LVTN/Dataset/LGG/Images"
path_mask_train = "C:/Users/sonbu/Desktop/LVTN/Dataset/LGG/Masks"

dataset = CustomDataset(img_dir = path_data_train, mask_dir = path_mask_train,transform = transforms.ToTensor())

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train, val, test = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(val, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

In [ ]:
def create_adj_matrix(size):
    
    col, row = np.meshgrid(np.arange(size), np.arange(size))
    coord = np.stack((col, row), axis=2).reshape(-1, 2)/ size
    dist = torch.from_numpy(cdist(coord, coord)).float().to(device)
    sigma_dist = torch.var(dist)

    A = torch.exp(-dist**2 / sigma_dist)
    
    D = torch.diag(torch.sum(A, axis=1))
    D_hat = torch.sqrt(torch.linalg.inv(D))

    A_hat =  torch.matmul(D_hat, torch.matmul(A, D_hat))
    
    return A_hat

adj = create_adj_matrix(size=img_size)

In [ ]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.projection = nn.Linear(in_features, out_features, bias=False)

    def forward(self, input, adj):
        batch_size = input.size(0)
        support = self.projection(input)
        output = torch.stack([torch.mm(adj, support[b]) for b in range(batch_size)])
        return output

In [ ]:
class GCNN(nn.Module):
    def __init__(self, img_size, in_features, adj):
        super().__init__()
        self.N = img_size ** 2
        self.adj = nn.Parameter(adj)
        self.gc1 = GraphConvolution(in_features, 16)
        self.gc2 = GraphConvolution(16, 32)
        self.gc3 = GraphConvolution(32, 64)
        self.gc4 = GraphConvolution(64, 128)
        self.gc5 = GraphConvolution(128, 256)
        self.gc6 = GraphConvolution(256, 1)

    def forward(self, x):
        x = F.relu(self.gc1(x, self.adj))
        x = F.relu(self.gc2(x, self.adj))
        x = F.relu(self.gc3(x, self.adj))
        x = F.relu(self.gc4(x, self.adj))
        x = F.relu(self.gc5(x, self.adj))
        x = self.gc6(x, self.adj)
        return torch.sigmoid(x)

In [ ]:
model = GCNN(img_size=img_size, in_features=1, adj = adj).to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_func = nn.BCELoss()

In [ ]:
train_losses, val_losses = [], []

for epoch in range(epochs):
    start = time.time()

    train_loss = 0.0  
    model.train()
    for batch_id, (data, label) in enumerate((train_loader)):
        batch_size = data.size(0)
        data, label = data.to(device), label.to(device)
        data = data.permute(0, 2, 3, 1).reshape(batch_size, -1, 1)
        label = label.reshape(batch_size, -1, 1)
        optimizer.zero_grad()
        pred = model(data)
        loss = loss_func(pred, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() / len(train_loader)

    train_losses.append(train_loss)

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch_id, (data, label) in enumerate((val_loader)):
            batch_size = data.size(0)
            data, label = data.to(device), label.to(device)
            data = data.permute(0, 2, 3, 1).reshape(batch_size, -1, 1)
            label = label.reshape(batch_size, -1, 1)
            pred = model(data)
            loss = loss_func(pred, label)
        
            val_loss += loss.item() / len(val_loader)

    val_losses.append(val_loss)

    end = time.time()
    
    print('Epoch: [{:4}/{:4}] - Train loss: {:.4f} - Val loss: {:.4f} - Time: {:.2f}'.format(epoch + 1, epochs, train_loss, val_loss, end - start))

In [ ]:
plt.plot(train_losses, label='Train loss')
plt.plot(val_losses, label='Val loss')
plt.legend()
plt.show()

In [ ]:
#torch.save(model.state_dict(), 'C:/Users/sonbu/Desktop/LVTN/Model/GCNN/GCNN_LGG.pt')

In [ ]:
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix, jaccard_score
test_accuracy, pre, sen, dsc, spe, iou = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
length_data = len(test_loader)

model.eval()

with torch.no_grad():
    for batch_id, (data, label) in enumerate(train_loader):
        
        batch_size = data.size(0)
        data, label = data.to(device), label.to(device)
        data = data.permute(0, 2, 3, 1).view(batch_size, -1, 1)
        label = label.view(batch_size, -1, 1)
        
        pred = model(data)
        loss = loss_func(pred, label)

        pred[pred > 0.5] = 1
        pred[pred <= 0.5] = 0
        lb = label
        lb[lb > 0.5] = 1
        lb[lb <= 0.5] = 0
       
    
        total_test= lb.numel()
        correct_test = pred.eq(lb).sum().item()
        test_accuracy += (float(correct_test*100) / total_test)

        precision,recall, f1_score, sup = precision_recall_fscore_support(lb.view(-1, 1).cpu().numpy(), pred.view(-1, 1).cpu().numpy(),zero_division=1, average='binary')
        cm = confusion_matrix(lb.view(-1, 1).cpu().numpy(), pred.view(-1, 1).cpu().numpy())
        specificity = cm[0,0]/(cm[0,0]+cm[0,1])
        iou_score = jaccard_score(lb.view(-1, 1).cpu().numpy(), pred.view(-1, 1).cpu().numpy())
        pre += precision
        sen += recall
        dsc += f1_score
        spe += specificity
        
        #print('IoU: {:.2f}%'.format(iou_score*100))
        
        plt.figure()
        plt.subplot(131)
        plt.title("Ảnh gốc")
        plt.imshow(data[0].view(img_size, img_size).cpu().detach().numpy())
        plt.subplot(132)
        plt.title("Ground Truth")
        plt.imshow(label[0].view(img_size, img_size).cpu().detach().numpy(), cmap='gray')
        plt.subplot(133)
        plt.title("Predicted")
        plt.imshow(pred[0].view(img_size, img_size).cpu().detach().numpy(), cmap='gray')
        plt.show()
    
    print('Accurcy: {:.2f} - DSC: {:.2f}% - PRE: {:.2f}% - SEN: {:.2f}% - SPE: {:.2f}%'.format(test_accuracy /length_data, dsc*100/length_data, pre*100/length_data, sen*100/length_data, spe*100/length_data))

        